# Fetch the data from the data source.

In [1]:
# Import the libraries

import os
import pandas as pd

In [2]:
# Get back to our project root directory

print(os.getcwd())

/home/jatin/Study/Live Courses/Projects/End-to-End-ML-Pipeline/research


In [3]:
# Go back to the project root directory.

os.chdir("../")
print(os.getcwd())

/home/jatin/Study/Live Courses/Projects/End-to-End-ML-Pipeline


In [4]:
# Read the data from the file

pd.read_csv('data/winequality_data.csv')

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
from dataclasses import dataclass
from pathlib import Path

# This is called entity and this is the return type of the configuration function.

@dataclass(frozen=True) # Why we are using data class and will it do.
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# This entity will return all the variable we have defined in the config.yaml

In [6]:
from ML_pipeline.constants import *
from ML_pipeline.utils.common import read_yaml, create_directories

In [7]:
# Define the configuration manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# Update the Components

In [8]:
# Import the necessary libraries

import os
import urllib.request as request
import zipfile
from ML_pipeline import logger
from ML_pipeline.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
# Create the pipeline

# i.e., Steps of execution

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-11 22:37:06,809: INFO: common: 2025-06-11 22:37:06,809]
[2025-06-11 22:37:06,814: INFO: common: 2025-06-11 22:37:06,814]
[2025-06-11 22:37:06,818: INFO: common: 2025-06-11 22:37:06,818]
[2025-06-11 22:37:06,821: INFO: common: 2025-06-11 22:37:06,821]
[2025-06-11 22:37:06,823: INFO: common: 2025-06-11 22:37:06,823]
[2025-06-11 22:37:07,999: INFO: 635984761: 2025-06-11 22:37:07,999]


- Bappy project Github

https://github.com/entbappy/End-to-End-Machine-Learning-Pipeline/blob/main/src/mlProject/constants/__init__.py